ref: https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py

In [1]:
import os
os.environ['KERAS_BACKEND']='theano'
#os.environ['THEANO_FLAGS']="floatX=float64,device=cpu"
os.environ['THEANO_FLAGS']="floatX=float32,device=cuda"

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

Using Theano backend.
Using cuDNN version 5105 on context None
Mapped name None to device cuda: GeForce GTX 1080 (0000:01:00.0)


取得尼采文字

In [3]:
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 600893


In [4]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))

total chars: 57


In [5]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [6]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 200285


Vectorization

In [7]:
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

The model

In [8]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, 
              metrics=['accuracy', 'top_k_categorical_accuracy'])

In [9]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [10]:
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]        
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char                    
        print(generated)


--------------------------------------------------
Iteration 1
Epoch 1/1
200285/200285 [==============================] - 37s - loss: 1.9964 - acc: 0.4176 - top_k_categorical_accuracy: 0.7684    

----- diversity: 0.2
----- Generating with seed: "rate himself before these opinions, as
t"
rate himself before these opinions, as
the sentent the sentent of the sense of the sentent the sense of the sense of the sense of the senting in the sense of the sentent the sentent the sentent the self-such the fain of the serfove the present the sense of still and the senting in the self-streat the sense of strang the serves and his oble the serves the sense of the self--and and the sense of string the self-such the self--and the sent

----- diversity: 0.5
----- Generating with seed: "rate himself before these opinions, as
t"
rate himself before these opinions, as
the consequence with in the serves of sensing how for ounterful become in the the the old an into
proposing, to the soul were in the perh